### Скачать сразу обработанный датасет можно по следующим ссылкам: 

Шаг 1 (Okutama-Action): используется для предварительного обучения https://disk.yandex.ru/d/CjuAiRn-OtOoPg

Шаг 2 (предоставленный набор): используется для дообучения https://disk.yandex.ru/d/UjmqQxbmU9QLcw

# Okutama-Action
Исходный набор данных можно скачать по ссылке http://okutama-action.org/. Я использовал тренировочный и тестовый набор с изображениями 1280x720 пикселей (5,3 и 1,5 ГБ).

Код в двух ячейках ниже нужен для подготовки набора Okutama-Action. Обе подвыборки будут использоваться для обучения YOLOv7-E6E.

In [1]:
'''
    Train set
'''

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from glob import glob
from tqdm import tqdm


paths = glob(r'F:\omsk\TrainSetFrames\*\*\*\*')
paths = [name for name in paths if not 'Labels' in name]
txt_train_path = r'F:\omsk\TrainSetFrames\Labels\MultiActionLabels\3840x2160'
txt_names = os.listdir(txt_train_path)
print(len(paths) == len(txt_names))

count = 0
dest = 'F:/omsk/data_for_yolo/data'
for k, path in enumerate(paths):
    label_path = f'F:/omsk/TrainSetFrames/Labels/MultiActionLabels/3840x2160/{txt_names[k]}'

    with open(label_path, 'r') as file:
        arr = file.read().split('\n')
        for i in range(len(arr)):
            arr[i] = arr[i].split(' ')[1:9]
            for j in range(len(arr[i])):
                if j < 4:
                    arr[i][j] = int(arr[i][j]) // 3
                else:
                    arr[i][j] = int(arr[i][j])
        del arr[-1]

    df = pd.DataFrame(arr, columns=['xmin', 'ymin', 'xmax', 'ymax', 'frame', 'lost', 'occluded', 'generated'])
    df = df.sort_values(by='frame')
    count += len(os.listdir(f'{dest}/labels')) + 100000
    for i in tqdm(range(len(df)), total=len(df), desc=f'{k}/{len(txt_names)}'):
        x = list(df.iloc[i])
        img_path = f'{path}/{x[4]}.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if (x[3] - x[1] > 15 or  x[2] - x[0] > 15) and x[5] == 0:
            xmin = x[0]
            xmax = x[2]
            ymin = x[1]
            ymax = x[3]

            l = max([xmax-xmin, ymax-ymin])

            shutil.copy(f'{path}/{x[4]}.jpg', f'{dest}/images/{x[4] + count}.jpg')

            if os.path.isfile(f'{dest}/labels/{x[4] + count}.txt'):
                with open(f'{dest}/labels/{x[4] + count}.txt', 'a') as file:
                    file.write(f'0 {(xmin + xmax) / 2 / 1280} {(ymin + ymax) / 2 / 720} {l / 1280} {l / 720}\n')
            else:           
                with open(f'{dest}/labels/{x[4] + count}.txt', 'w') as file:
                    file.write(f'0 {(xmin + xmax) / 2 / 1280} {(ymin + ymax) / 2 / 720} {l / 1280} {l / 720}\n')

    #         plt.imshow(image[xmin:xmax, ymin:ymax])
    #         plt.show()

True


32/33: 100%|██████████| 11584/11584 [01:22<00:00, 140.07it/s]


In [2]:
'''
    Test set
'''

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from glob import glob
from tqdm import tqdm


paths = glob(r'F:\omsk\TestSetFrames\*\*\*\*')
paths = [name for name in paths if not 'Labels' in name]
txt_train_path = r'F:\omsk\TestSetFrames\Labels\MultiActionLabels\3840x2160'
txt_names = os.listdir(txt_train_path)
print(len(paths) == len(txt_names))

count = 0
dest = 'F:/omsk/data_for_yolo/data2'
for k, path in enumerate(paths):
    label_path = f'F:/omsk/TestSetFrames/Labels/MultiActionLabels/3840x2160/{txt_names[k]}'

    with open(label_path, 'r') as file:
        arr = file.read().split('\n')
        for i in range(len(arr)):
            arr[i] = arr[i].split(' ')[1:9]
            for j in range(len(arr[i])):
                if j < 4:
                    arr[i][j] = int(arr[i][j]) // 3
                else:
                    arr[i][j] = int(arr[i][j])

        del arr[-1]

    df = pd.DataFrame(arr, columns=['xmin', 'ymin', 'xmax', 'ymax', 'frame', 'lost', 'occluded', 'generated'])
    df = df.sort_values(by='frame')
    count += len(os.listdir(f'{dest}/labels')) + 100000
    for i in tqdm(range(len(df)), total=len(df), desc=f'{k}/{len(txt_names)}'):
        x = list(df.iloc[i]) 
        img_path = f'{path}/{x[4]}.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if (x[3] - x[1] > 15 or  x[2] - x[0] > 15) and x[5] == 0:
            xmin = x[0]
            xmax = x[2]
            ymin = x[1]
            ymax = x[3]

            l = max([xmax-xmin, ymax-ymin])

            shutil.copy(f'{path}/{x[4]}.jpg', f'{dest}/images/{x[4] + count}.jpg')

            if os.path.isfile(f'{dest}/labels/{x[4] + count}.txt'):
                with open(f'{dest}/labels/{x[4] + count}.txt', 'a') as file:
                    file.write(f'0 {(xmin + xmax) / 2 / 1280} {(ymin + ymax) / 2 / 720} {l / 1280} {l / 720}\n')
            else:           
                with open(f'{dest}/labels/{x[4] + count}.txt', 'w') as file:
                    file.write(f'0 {(xmin + xmax) / 2 / 1280} {(ymin + ymax) / 2 / 720} {l / 1280} {l / 720}\n')

    #         plt.imshow(image[xmin:xmax, ymin:ymax])
    #         plt.show()

True


9/10: 100%|██████████| 5497/5497 [00:37<00:00, 146.78it/s]


# Предоставленный набор
В качестве тестовой выборки будут использоваться изображения с людьми из предоставленного набора.

In [2]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt

data_df = pd.read_csv("F:/omsk/train/train.csv")
data_df

C:\Users\mmill\anaconda3\envs\torch_121\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mmill\anaconda3\envs\torch_121\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\mmill\anaconda3\envs\torch_121\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


,ID_img,count_region,region_shape
0,3436.JPG,0.0,0.0
1,3437.JPG,0.0,0.0
2,3438.JPG,0.0,0.0
3,3439.JPG,0.0,0.0
4,3440.JPG,0.0,0.0
...,...,...,...
5157,8593.JPG,0.0,0.0
5158,8594.JPG,0.0,0.0
5159,8595.JPG,0.0,0.0
5160,8596.JPG,0.0,0.0


In [ ]:
import os

l = data_df.loc[data_df['count_region'] != 0.0]['ID_img']

for img_name in l:
    s = list(data_df.loc[data_df['ID_img'] == img_name]['region_shape'])[0]
    s2 = s.split(':')
    for i in range(len(s2)):
        s2[i] = s2[i][:4]
        if s2[i] != '':
            count = 0
            while s2[i][count].isdigit():
                count += 1
                if count == 4:
                    break
            s2[i] = s2[i][:count]

        if s2[i] != '':     
            s2[i] = int(s2[i])
    del s2[0]    
    
    image = cv2.imread(f'F:/omsk/train/train/{img_name}')
    size = image.shape[:2]
    for i in range(0, len(s2), 3):
        xmin = s2[i+1]-s2[i+2]
        xmax = s2[i+1] + s2[i+2]
        ymin = s2[i] -s2[i+2] 
        ymax = s2[i] +s2[i+2]

        if xmin < 0:
            xmin = 0
        if ymin < 0:
            ymin = 0
#         plt.imshow(image[xmin:xmax, ymin:ymax])
#         plt.show()
        path_txt = f'F:/omsk/train/{img_name[:-4]}.txt'
        if os.path.isfile(path_txt):
            with open(path_txt, 'a') as file:
                file.write(f'0 {s2[i] / size[1]} {s2[i+1] / size[0]} {s2[i+2] / size[1]} {s2[i+2] / size[0]}\n')
        else:
            with open(path_txt, 'w') as file:
                file.write(f'0 {s2[i] / size[1]} {s2[i+1] / size[0]} {s2[i+2] / size[1]} {s2[i+2] / size[0]}\n')

In [ ]:
import shutil

for name in l:
    shutil.copy(f'F:/omsk/train/train/{name}', f'F:/omsk/train/{name}')